In [1]:
# Load libraries
# !pip install -r /work/NLP_IMDb_Exam/requirements.txt
import numpy as np
import pandas as pd
import torch
import datasets
import evaluate
import seaborn as sns
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModel
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from sentence_transformers import SentenceTransformer

/home/ucloud/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-22 19:06:26.846199: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734890786.859248   10457 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734890786.863253   10457 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-22 19:06:26.879225: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operatio

In [2]:
# Read embeddings from CSV
data_path = '../Data/IMDB_Embeddings_review_instruct_classifying.csv'
emb_full=pd.read_csv(data_path)
emb_full = emb_full.drop(columns=['Unnamed: 0']) # Drop index collumn
emb_full

,0,1,2,3,4,5,6,7,8,9,...,761,762,763,764,765,766,767,rating,average_rating,review
0,-0.049986,-0.003046,-0.005113,0.014502,0.031239,0.007906,-0.049414,0.017113,-0.018944,0.039872,...,-0.002763,0.013105,-0.043225,-0.044028,-0.047788,0.016681,0.043883,7,7.2,This movie is full of suspense. It makes you g...
1,-0.052491,0.001266,-0.009839,0.027879,0.051474,0.022958,-0.018116,0.000579,-0.009751,0.037017,...,-0.006076,-0.014107,-0.016305,-0.031785,-0.061127,0.010477,0.041518,5,7.2,I'll leave this review fairly concise. <br/><b...
2,-0.077953,0.012166,-0.012654,0.037254,0.027758,0.046309,-0.032994,0.009308,0.012343,0.044122,...,-0.022567,-0.011914,-0.029081,-0.024161,-0.060691,0.012708,0.039456,5,7.2,I give the 5/10 out of the credit I owe to the...
3,-0.062827,-0.000282,-0.011925,0.012841,0.049001,0.027900,-0.029636,-0.003337,-0.026158,0.042310,...,-0.018817,0.001601,-0.037072,-0.054246,-0.056599,-0.002461,0.048593,5,7.2,"First of all, I must say that I was expecting ..."
4,-0.072137,-0.002611,-0.027151,0.039950,0.064210,0.023805,-0.016661,0.038756,0.006763,0.053410,...,0.006199,-0.007624,-0.046628,-0.033239,-0.036501,0.024232,0.043796,7,7.2,I've always loved movies with strong atmospher...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,-0.059426,-0.003312,-0.007381,0.036839,0.051791,0.009148,-0.025224,0.010144,0.017306,0.048820,...,-0.006785,-0.002487,-0.024572,-0.037381,-0.064394,0.024573,0.012697,8,7.7,SPOILER Being John Malkovich is a thought-prov...
99996,-0.053319,-0.016404,-0.003469,0.045656,0.059463,0.026696,-0.042109,0.022793,0.003686,0.052823,...,-0.010148,0.010981,-0.078704,-0.053920,-0.065344,0.032776,0.026892,8,7.7,i didn't know what to expect from this movie. ...
99997,-0.053106,-0.012624,-0.007986,0.041956,0.047385,0.033586,-0.015612,0.020658,-0.002866,0.015201,...,-0.017430,0.013684,-0.056483,-0.044315,-0.045562,0.035103,0.034681,5,7.7,This is the first time in a good while that I'...
99998,-0.043777,-0.011234,-0.008850,0.054473,0.040372,0.024090,-0.024706,0.030505,0.007630,0.043525,...,-0.023170,-0.010481,-0.047681,-0.042667,-0.063741,0.036404,0.032964,10,7.7,I admit that I wanted to see this film for the...


In [3]:
from sklearn.model_selection import train_test_split
# Remove rest of unecessary columns before splitting:
split = emb_full.drop(columns=['review'])
split = split.drop(columns=['rating'])
split = train_test_split(split, train_size=0.7)
finaleval=split[1]
subset=split[0]

train_features = subset.drop(columns=["average_rating"]).to_numpy()
train_labels = np.float32(subset['average_rating'].to_numpy())
eval_features = finaleval.drop(columns=["average_rating"]).to_numpy()
eval_labels = np.float32(finaleval[['average_rating']].to_numpy())

# Create a linear regression of entire embedding space:

In [4]:
# !pip install statsmodels
import statsmodels.api as sm
#Regression Model
X = sm.add_constant(train_features)# adding a constant
model=sm.OLS(train_labels, X).fit()

In [5]:
# Predictive accuracy:
X_test = sm.add_constant(eval_features) # add intercept to test_data
predictions = model.predict(X_test) # predict

In [6]:
# Summary Statistics
print("R-squared:", model.rsquared)
eval_labels = eval_labels.ravel()
results_df = pd.DataFrame({
    'True Labels': eval_labels,
    'Predicted Labels': predictions
})
print("MSE:", np.mean((results_df['True Labels'] - results_df['Predicted Labels'])**2))
print("Corr:", results_df.corr().iloc[0,1])
results_df

R-squared: 0.650082802630465
MSE: 0.21055802968863352
Corr: 0.7974684523595609


,True Labels,Predicted Labels
0,8.9,8.646195
1,6.8,7.223224
2,7.3,8.016779
3,8.4,7.963231
4,8.4,7.711352
...,...,...
29995,7.3,8.017656
29996,7.2,7.329712
29997,8.4,7.773644
29998,8.3,8.225047


# Create an ICA model:

In [7]:
from sklearn.decomposition import FastICA
ica = FastICA(n_components=20, whiten="arbitrary-variance") # Define ICA
My_ICA_fit = ica.fit_transform(train_features) # Fit ICA to data

/home/ucloud/.local/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


In [8]:
# Get estimated mixing matrix
mixMatrixEst = ica.mixing_

# Map embeddings onto ICA-space
train_ICA_features = pd.DataFrame(train_features @ mixMatrixEst)
eval_ICA_features = pd.DataFrame(eval_features @ mixMatrixEst)

In [9]:
# !pip install statsmodels
import statsmodels.api as sm
#Regression Model
X = sm.add_constant(train_features)# adding a constant
model=sm.OLS(train_labels, X).fit()

In [10]:
# Predictive accuracy:
X_test = sm.add_constant(eval_features) # add intercept to test_data
predictions = model.predict(X_test) # predict

In [12]:
# Summary Statistics
print("R-squared:", model.rsquared)
eval_labels = eval_labels.ravel()
results_df = pd.DataFrame({
    'True Labels': eval_labels,
    'Predicted Labels': predictions
})
print("MSE:", np.mean((results_df['True Labels'] - results_df['Predicted Labels'])**2))
print("Corr:", results_df.corr().iloc[0,1])
results_df

R-squared: 0.650082802630465
MSE: 0.21055802968863352
Corr: 0.7974684523595609


,True Labels,Predicted Labels
0,8.9,8.646195
1,6.8,7.223224
2,7.3,8.016779
3,8.4,7.963231
4,8.4,7.711352
...,...,...
29995,7.3,8.017656
29996,7.2,7.329712
29997,8.4,7.773644
29998,8.3,8.225047
